# **Francisco del Val Yagüe**

------------------------

# TAREA FINAL: Datos de vivienda

<p> El negocio inmobiliario es uno de los motores de la economía de España, el carácter turístico de nuesta península hace que sean muchos los inversores extranjeros que se decidan por buscar una inversión de bajo riesgos en nuestras ciudades costeras o grandes urbes. 
Muchas de estas inversiones se hacen sobre grandes bolsas de inmuebles que deben ser analizados previamente para comprobar la rentabilidad del porfolio </p>

<!-- <p> En este caso vamos a trabajar con una tabla que contienen información de distintos inmuebles repartidos por una zona específica, sus carácterísticas y su precio </p>  -->

Todas las cuestiones se deben realizar sobre el conjunto de casos que representen viviendas ubicadas en zonas residenciales **(alta, media y baja densidad)**

**MUY IMPORTANTE:** En las otras prácticas he detectado colaboraciones involucrando varias personas y he sido flexible aunque a algunos os lo he mencionado en las correcciones, porque al final el trabajo de analista de datos es un trabajo colaborativo. Sin embargo, este trabajo es individual, así que cuidado con las colaboraciones.

Las variables de las que se compone el dataset son:

|NOMBRE VARIABLE|DESCRIPTOR|VALORES|
| --- | --- | --- |
|Order|Variable de identificación|1 a 2930|
|MS Zoning|Zona de ubicación de la vivienda|"A rural, C comercial, FV residencial flotante, I industrial, RH residencial alta densidad, RL residencial baja densidad, RM residencial media densidad"|
|Lot Frontage|Longitud de la fachada en pies||
|Lot Area|Superficie de la vivienda en pies cuadrados||
|Land Contour|Contorno del terreno circundante|"Lvl llano, Bnk Tipo bancal, HLS Ladera, Low Depresión"|
|Land Slope|Tipo de pendiente de la vivienda|" Gtl pendiente suave, Mod pendiente moderada, Sev fuerte pendiente"|
|Overall Qual|Grado de calidad de materiales y acabado de la vivienda|De 1 (Muy pobre) a 10 (Excelente)|
|Year Built|Año de construccion de la vivienda||
|Year Remod/Add|Año de última reforma de la vivienda||
|Mas Vnr Type|Tipo de revestimiento exterior|" BrkCmn Ladrillo normal, BrkFace Ladrillo visto, CBlock Bloque de cemento, None Ninguna, Stone Piedra "|
|Exter Qual|Calidad de revestimiento exterior|"Ex Excelente,Gd Bueno,TA Media,Fa Justo"|
|Bsmt Cond|Estado general del sótano|"Ex Excelente, Gd Bueno, TA Media, Fa Justo, Po Pobre,Ss sin sótano"|
|Total Bsmt SF|Superficie del sótano en pies cuadrados|
|Heating QC|Calidad de la calefacción|"Ex Excelente,Gd Bueno,TA Media,Fa Justo,Po Pobre"|
|Central Air|Aire acondicionado centralizado|"N No Y Sí"|
|Full Bath|Número de baños completo en planta||
|Half Bath|Número de aseos en planta||
|Bedroom AbvGr|Número de dormitorios en planta||
|Kitchen AbvGr|Número de cocinas en planta||
|Kitchen Qual|Calidad de cocinas|"Ex Excelente,Gd Bueno,TA Media,Fa Justo,Po Pobre"|
|TotRms AbvGrd|Número total de habitaciones excluidos los cuartos de baño||
|Garage Cars|Número de plazas de garaje||
|Garage Area|Superficie del garaje|||
|Garage Cond|Estado del garaje|"Ex Excelente,Gd Bueno,TA Media,Fa Justo,Po Pobre,Sg sin garaje"|
|Pool Area|Superficie de la piscina en pies cuadrados|
|Pool QC|Calidad de la piscina|"Ex Excelente,Gd Bueno,TA Media,Fa Justo,Sp no hay piscina"|
|Mo Sold|mes de venta||
|Yr Sold|año de venta||
|SalePrice|precio de venta en dólares||

Recomiendo al leer los datos, eliminar espacios de los nombres de las columnas, realiza un pequeño análisis inicial de los mismos. No olvides fijarte en los tipos de variables, que variables pueden tener tipos confundidos y corrige los. Sobre todo, trabaja con las fechas.

**NOTA:** Las tareas complementarias sirven para subir nota. El resto de preguntas valen igual y suman 10 puntos.

-------

## **1. Inicializar y cargar el contexto spark**

In [ ]:
# Instalacion
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark

# Importo un enviroment de java y spark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

# Impoero e inicio findspark
import findspark
findspark.init()

# Inicio sesion del contexto spark
from pyspark import SparkContext

#Creo un objeto rdd para trabajar con el
sc = SparkContext.getOrCreate()

# Importo SparkSession desde pyspark.sql
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

# Creacion de objeto llamado spark
spark = SparkSession.builder.master("local[*]").getOrCreate()

### **Carga Base de Datos** 

1. Carga y Visualización

In [ ]:
data_file_vivienda = "BDpracticafinalCSV.csv"
raw_vivienda = sc.textFile(data_file_vivienda) # leemos el archivo 
raw_vivienda.take(2) # compruebo que se ha cargado bien y cual es el separador

['Order;MS Zoning;Lot Frontage;Lot Area;Land Contour;Land Slope;Overall Qual;Year Built;Year Remod/Add;Mas Vnr Type;Exter Qual;Bsmt Cond;Total Bsmt SF;Heating QC;Central Air;Full Bath;Half Bath;Bedroom AbvGr;Kitchen AbvGr;Kitchen Qual;TotRms AbvGrd;Garage Cars;Garage Area;Garage Cond;Pool Area;Pool QC;Mo Sold;Yr Sold;SalePrice',
 '1;RL;141;31770;Lvl;Gtl;6;1960;1960;Stone;TA;Gd;1080;Fa;Y;1;0;3;1;TA;7;2;528;TA;0;Sp;5;2010;215000']

In [ ]:
# Cuento el numero de registros en total
raw_vivienda.count()

2937

2. Eliminación de Cabecera

In [ ]:
cabecera = raw_vivienda.first()
raw_vivienda_sin_cabecera = raw_vivienda.filter(lambda x: x != cabecera)
raw_vivienda_sin_cabecera.take(2)

['1;RL;141;31770;Lvl;Gtl;6;1960;1960;Stone;TA;Gd;1080;Fa;Y;1;0;3;1;TA;7;2;528;TA;0;Sp;5;2010;215000',
 '2;RH;80;11622;Lvl;Gtl;5;1961;1961;None;TA;TA;882;TA;Y;1;0;2;1;TA;5;1;730;TA;0;Sp;6;2010;105000']

3. Separación por `","`

In [ ]:
vivienda_pars = raw_vivienda_sin_cabecera.map(lambda x: x.split(";"))
vivienda_pars.take(2)

[['1',
  'RL',
  '141',
  '31770',
  'Lvl',
  'Gtl',
  '6',
  '1960',
  '1960',
  'Stone',
  'TA',
  'Gd',
  '1080',
  'Fa',
  'Y',
  '1',
  '0',
  '3',
  '1',
  'TA',
  '7',
  '2',
  '528',
  'TA',
  '0',
  'Sp',
  '5',
  '2010',
  '215000'],
 ['2',
  'RH',
  '80',
  '11622',
  'Lvl',
  'Gtl',
  '5',
  '1961',
  '1961',
  'None',
  'TA',
  'TA',
  '882',
  'TA',
  'Y',
  '1',
  '0',
  '2',
  '1',
  'TA',
  '5',
  '1',
  '730',
  'TA',
  '0',
  'Sp',
  '6',
  '2010',
  '105000']]

4. Paso a float los valores perdidos

In [ ]:
def tofloat (x):
  if x in ["" , " " , "nan" , "null"]:
    output = 0.0
  else:
    output = float(x)
  return output

5. Aplicación función

Se pretende convertir todos los valores perdidos que se encuentran en la base de dados a float, con la finalidad de que no exista ningun error y mas adelante arrastremos el mimso y no podamos trabajar correctamente. 

-------

## **2. Cuántas viviendas distintas encontramos en el dataset? ¿Se repite alguna? Tiene sentido que haya duplicadas? ¿Qué podemos hacer con las duplicadas?**

In [ ]:
print("El número de viviendas distintas es: {} ".format(vivienda_pars.map(lambda x: x[0]).distinct().count()))

El número de viviendas distintas es: 2930 


Como podemos observar, sí existen duplicados. Ya que, al contar al princio el total de registros, vemos que suman 2937 y después de hacer el distinct obtenemos 2930 viviendas, por lo que existen 7 viviendas repetidas. Como es un número muy reducido en comparacion con la muestra, trabajaremos con ello. No obstante, se podría usar dataframe y borrarlos con drop_duplicated. Pero en nuestro caso trabajamos con RDD y decidimos mantenerlos. 

-----------

## **3. ¿Podrías decirme el total de inmuebles y el precio medio (Sale Price) de cada zona (MS Zoning)?**

In [ ]:
# 1. Zonas que existen
vivienda_pars.map(lambda x: x[1]).distinct().collect()

['RL', 'RH', 'C', 'Rl', 'FV', 'RM', 'I', 'A', 'rL']

In [ ]:
# 2. Seleccionamos las distintas zonas x[1] : 'RL', 'RH', 'C', 'Rl', 'FV', 'RM', 'I', 'A', 'rL'
# y precio x[28]
RL = vivienda_pars.filter(lambda x: x[1]== 'RL').map(lambda x: (x[0],x[28]))
RH = vivienda_pars.filter(lambda x: x[1]== 'RH').map(lambda x: (x[0],x[28]))
C = vivienda_pars.filter(lambda x: x[1]== 'C').map(lambda x: (x[0],x[28]))
Rl = vivienda_pars.filter(lambda x: x[1]== 'Rl').map(lambda x: (x[0],x[28]))
FV = vivienda_pars.filter(lambda x: x[1]== 'FV').map(lambda x: (x[0],x[28]))
RM = vivienda_pars.filter(lambda x: x[1]== 'RM').map(lambda x: (x[0],x[28]))
I = vivienda_pars.filter(lambda x: x[1]== 'I').map(lambda x: (x[0],x[28]))
A = vivienda_pars.filter(lambda x: x[1]== 'A').map(lambda x: (x[0],x[28]))
rL = vivienda_pars.filter(lambda x: x[1]== 'rL').map(lambda x: (x[0],x[28]))

In [ ]:
# 3. Número de viviendas por cada zona
print("El número de viviendas RL es: {} ".format(RL.count()))
print("El número de viviendas RH es: {} ".format(RH.count()))
print("El número de viviendas C es: {} ".format(C.count()))
print("El número de viviendas Rl es: {} ".format(Rl.count()))
print("El número de viviendas FV es: {} ".format(FV.count()))
print("El número de viviendas RM es: {} ".format(RM.count()))
print("El número de viviendas I es: {} ".format(I.count()))
print("El número de viviendas A es: {} ".format(A.count()))
print("El número de viviendas rL es: {} ".format(rL.count()))


El número de viviendas RL es: 2271 
El número de viviendas RH es: 27 
El número de viviendas C es: 25 
El número de viviendas Rl es: 6 
El número de viviendas FV es: 139 
El número de viviendas RM es: 462 
El número de viviendas I es: 2 
El número de viviendas A es: 2 
El número de viviendas rL es: 2 


In [ ]:
# 4. Precio medio por zona
Precio_RL = Rl.map(lambda x: tofloat(x[1])).mean()
Precio_RH = RH.map(lambda x: tofloat(x[1])).mean()
Precio_C = C.map(lambda x: tofloat(x[1])).mean()
Precio_Rl = Rl.map(lambda x: tofloat(x[1])).mean()
Precio_FV = FV.map(lambda x: tofloat(x[1])).mean()
Precio_RM = RM.map(lambda x: tofloat(x[1])).mean()
Precio_I = I.map(lambda x: tofloat(x[1])).mean()
Precio_A = A.map(lambda x: tofloat(x[1])).mean()
Precio_rL = rL.map(lambda x: tofloat(x[1])).mean()

In [ ]:
print("El precio medio de RL es: {} ".format(Precio_RL))
print("El precio medio de RH es: {} ".format(Precio_RH))
print("El precio medio de C es: {} ".format(Precio_C))
print("El precio medio de Rl es: {} ".format(Precio_Rl))
print("El precio medio de FV es: {} ".format(Precio_FV))
print("El precio medio de RM es: {} ".format(Precio_RM))
print("El precio medio de I es: {} ".format(Precio_I))
print("El precio medio de A es: {} ".format(Precio_A))
print("El precio medio de rL es: {} ".format(Precio_rL))

El precio medio de RL es: 254833.33333333334 
El precio medio de RH es: 136419.77777777778 
El precio medio de C es: 79795.04 
El precio medio de Rl es: 254833.33333333334 
El precio medio de FV es: 218986.94964028776 
El precio medio de RM es: 126781.39393939397 
El precio medio de I es: 80312.5 
El precio medio de A es: 47300.0 
El precio medio de rL es: 320000.0 


Aunque el enuncido nos indica solo tres zonas, nos resulta mas interesante comparar todas las zonas, llegando a la conclusion que la zona con mayor precio es RL.

-------------

## **4. Media de Total Bsmt SF por cada década de construcción calculada a partir de Year Built.**
## **¿Cuál es la decada de construcción con viviendas mejor acondicionadas para el frío (Heating QC)?**

In [ ]:
# Vemos el numero de años y por lo tanto habrá 15 decadas
vivienda_pars.map(lambda x: (x[7])).distinct().collect()

In [ ]:
# Funcion decada
def decada(x):
    return x[7] + "0s"

In [ ]:
decada("2007-2008")

'00s'

In [ ]:
decada = vivienda_pars.filter(lambda x: x[13]=="Ex").map(lambda x: (decada(x[0], x[1]))).\
                                                    map(lambda x: (x[7],x[12]).reduceByKey(lambda x,y : x+y).sortBy(lambda x: -x[0])
decada.take(3) 

In [ ]:
vivienda_pars.filter(lambda x: x[13]=="Ex").map(lambda x: (x[0],x[7],x[12],x[13])).takeOrdered(1, lambda x: -float(x[1]))


[('18', '2010', '1856', 'Ex')]

En primer lugar, etiquetamos cada construccion con su década.Se filtra por calidad de calefaccion excelente, seleccionando las columnas Order, Year Built, Total Bsmt SF y Heating QC, y se ha ordenado de forma descendente por el año de costrucción. Por lo tanto, el año de construcción de viviendas mejor acondicionadas para el frío es 2010 perteneciente a la decada del 2000. Deducimos que los nuevos materiales evolucionan al paso del tiempo, lo que conlleva a un mejor material aislante para el frio, por lo tanto, nuestro resultado tiene sentido.

-----------

## **5. ¿Cuáles son las 10 viviendas que se vendieron por un precio más elevado por metro cuadrado en el año 2009?**

In [ ]:
# Metros cuadrados
metros = viviendas.map(lambda x: (tofloat(x[2]*0.09()))) 
metros

Las medidas cuadradas de la vivienda vienen dadas en pies, por lo tanto, tenemos que pasarlo a metros cuadrados. Esta acción se hace multiplicando por 0.09. Para nuestro análisis decidimos trabajar con la medida de pies puesto que no hemos conseguido convertirlo a metros cuadrados.  

In [ ]:
 viviendas = vivienda_pars.filter(lambda x: x[7]=='2009').map(lambda x: (x[0],x[1],x[3],x[7],x[-1])).takeOrdered(10,lambda x: -float(x[-1]))
 viviendas

[('45', 'RL', '12919', '2009', '611657'),
 ('431', 'RL', '12220', '2009', '402861'),
 ('428', 'RL', '12350', '2009', '398800'),
 ('39', 'RL', '10159', '2009', '395192'),
 ('37', 'RL', '12858', '2009', '376162'),
 ('522', 'FV', '9187', '2009', '370878'),
 ('819', 'RL', '10933', '2009', '323262'),
 ('242', 'RL', '17485', '2009', '308030'),
 ('40', 'RL', '12883', '2009', '290941'),
 ('319', 'RL', '12003', '2009', '269500')]

En primer lugar, hemos filatrado por el año 2009 y seleccionando las columnas de Order, MS Zoning, Year Built, SalePrice y ordenando de forma descendente por el precio. Consiguiendo de esta forma el precio más alto por metro cuadrado en el año 2009.


-----------

## **6. Media anual por zonas del precio de venta y metros cuadrados.**

In [ ]:
zona = vivienda_pars.filter(lambda x: (x[3],x[7])).map(lambda x: (x[1],x[7],x[-1], x[3])).collect()
zona

Se filtra por Lot Area y MS Year Built, y seleccionamos las columnas MS Zoning,Year Built, SalePrice y Lot Area para conseguir los metros cuadrados y precios, segun el año y zona de la vivienda. En este caso, tambien se debería haber paso la medida a metros cuadrados pero, como en el caso anterior, no se ha conseguido y por lo tanto, se decido trabjar en pies. 

-------------------

## **7. ¿Podrías decirme el total de recaudación de las casas de revistimiento (Mas Vnr Type) de piedra con respecto a las de ladrillo? ¿Hay diferencia significativa?**

Para realizar este ejercicio, calcularemos el máximo por tipo de vivienda para hacernos una idea del resultado. Y posteriormente, calcularemos el total de cada tipo de piedra para comprobar que estamos en lo cierto. 

* **Piedra**

In [ ]:
vivienda_pars.filter(lambda x: x[9] == 'Stone').map(lambda x: (x[-1],x[9])).reduceByKey(lambda y,z: y + z).collect()

In [ ]:
# Maximo
vivienda_pars.filter(lambda x: x[9] == 'Stone').map(lambda x: (x[-1],x[9])).takeOrdered(2, lambda x: -float(x[0]))

[('615000', 'Stone'), ('611657', 'Stone')]

In [ ]:
# Total
piedra = vivienda_pars.filter(lambda x: x[9] == 'Stone').map(lambda x: (x[0], x[9], x[-1]))
precio_piedra = piedra.map(lambda x: tofloat(x[2])).sum()

In [ ]:
print('El precio en total de las viviendas de piedra es:', precio_piedra )

El precio en total de las viviendas de piedra es: 65196277.0


* **Ladrillo**

In [ ]:
vivienda_pars.filter(lambda x: x[9] == 'BrkFace').map(lambda x: (x[-1],x[9])).reduceByKey(lambda y,z: y + z).collect()

In [ ]:
# Maximo
vivienda_pars.filter(lambda x: x[9] == 'BrkFace').map(lambda x: (x[-1],x[9])).takeOrdered(2, lambda x: -float(x[0]))

[('755000', 'BrkFace'), ('625000', 'BrkFace')]

In [ ]:
# Total
ladrillo = vivienda_pars.filter(lambda x: x[9] == 'BrkFace').map(lambda x: (x[0], x[9], x[-1]))
precio_ladrillo = ladrillo.map(lambda x: tofloat(x[2])).sum()

In [ ]:
print('El precio en total de las viviendas de ladrillo es:', precio_ladrillo )

El precio en total de las viviendas de ladrillo es: 186003261.0


Sí, existe diferencia signficativa. Las viviendias construidas con ladrillo son mas caras que las construidas con piedra. El precio máximo de las viviendas construidas con piedra es 615000 , mientras que las de ladrillo un máximo de 755.000. No obstante, se ha calculado el total recaudado de cada tipo de vivienda, consiguiendo una mayor recaudacion la de ladrillo que la de piedra. Esto no es significativo puesto que la base de datos puede haber recogido mas información de casas de ladrillo que de piedra.

-----------------------

## **8. ¿Cuánto son más caras las viviendas con 2 cocinas, con 2 o más plazas de garaje que las que tienen 1 cocina y 1 plaza de garaje? Comparar medias y cuartiles de ambos casos**

In [ ]:
vivienda_pars.filter(lambda x: (x[18] == 2) and (x[21] == 3)).map(lambda x: (x[0],x[-1])).reduceByKey(lambda y,z: y + z).collect()

Cuartil


In [ ]:
# Creo el objeto vivienda con el precio 
vivienda= vivienda_pars.map(lambda x: (int(x[-1]))).collect()
vivienda.sort()

Cuartiles 25% - 50% - 75% :

In [ ]:
print("El cuartil 25% es: {} ".format(vivienda[int(len(vivienda)*.25)]))
print("El cuartil 50% es: {} ".format(vivienda[int (len(vivienda)*.50)]))
print("El cuartil 75% es: {} ".format(vivienda[int (len(vivienda)*.75)]))

El cuartil 25% es: 129500 
El cuartil 50% es: 160250 
El cuartil 75% es: 213750 


Función cuartiles para clasificarlos:

In [ ]:
def cuartil(x):
    if x <= 129500:
      return (1)
    elif x <= 160250:
      return (2) 
    else:
      return (3)

Viviendas con 2 cocinas, con 2 o más plazas de garaje.

In [ ]:
vivienda_pars.filter(lambda x: (int(x[18] == 2),(x[21] == 3))).\
              map(lambda x: (cuartil(int(x[-1])),1)).reduceByKey(lambda y,z: y + z).collect()

[(2, 730), (3, 1467), (1, 739)]

Viviendas con 1 cocina y 1 plaza de garaje.

In [ ]:
vivienda_pars.filter(lambda x: (int(x[18] == 1),(x[21] == 1))).\
              map(lambda x: (cuartil(int(x[-1])),1)).reduceByKey(lambda y,z: y + z).collect()

[(2, 730), (3, 1467), (1, 739)]

A priori, se podría pensar que una vivienda con más cocinas y mas número de plazas de garaje podría existir una diferencia significativa en el precio. 
Después de realizar los calculos, observamos que no existe variación alguna en el precio de los dos supuestos casos de vivienda, clasificandolos en misma proporción.

-------------------------------

## **9. (COMPLEMENTARIA) Estudiar la relación entre el precio y el número de Garajes. (Recomiendo segmentar precio por cuartiles y estudiar el número de casos coincidentes)**

In [ ]:
# Creo el objeto vivienda con el precio y numero de garajes
vivienda= vivienda_pars.map(lambda x: (int(x[-1]),(x[21]))).collect()
vivienda.sort()

In [ ]:
# Calculo cuartiles
print("El cuartil 25% es: {} ".format(vivienda[int(len(x)*.25)]))
print("El cuartil 50% es: {} ".format(vivienda[int (len(x)*.50)]))
print("El cuartil 75% es: {} ".format(vivienda[int (len(x)*.75)]))

El cuartil 25% es: (129500, '1') 
El cuartil 50% es: (160250, '2') 
El cuartil 75% es: (213750, '2') 


In [ ]:
# funcion cuartil
def cuartil(x):
    if x <= 129500:
      return (1)
    elif x <= 160250:
      return (2) 
    else:
      return (3)

In [ ]:
vivienda_pars.filter(lambda x: (int(x[-1]),(x[21]))).map(lambda x: (cuartil(int(x[-1])),1)).\
reduceByKey(lambda y,z: y + z).collect()

[(2, 730), (3, 1467), (1, 739)]

Como podemos observar, podemos clasificar las viviendas por numero de garajes, existiendo un mayor numero de casas en el grupo 3 donde corresponden aquellas casas mas caras y por tanto mayor número de garajes. Por lo tanto, se afirma que sí existe correlación entre precio y número de garajes. 

-------------------

## **10. (COMPLEMENTARIA) Las 10 viviendas con mejores servicios y mejor precio.**

In [ ]:
vivienda_pars.filter(lambda x: ((x[13] == 'Ex'), (x[15] == 3),(x[16] == 3), (x[17] == 4), (x[18] == 2))).\
              map(lambda x: (x[7],x[14],x[16],x[17],x[18],x[-1])).takeOrdered(10, lambda x: -float(x[-1]))

[('1994', 'Y', '1', '4', '1', '755000'),
 ('1996', 'Y', '1', '4', '1', '745000'),
 ('1995', 'Y', '1', '4', '1', '625000'),
 ('2003', 'Y', '1', '1', '1', '615000'),
 ('2009', 'Y', '1', '2', '1', '611657'),
 ('2007', 'Y', '1', '2', '1', '610000'),
 ('2006', 'Y', '1', '4', '1', '591587'),
 ('1993', 'Y', '1', '4', '1', '584500'),
 ('2008', 'Y', '1', '4', '1', '582933'),
 ('2005', 'Y', '1', '4', '1', '556581')]

En este caso, se ha filtrado por excelente y por las caracteristicas de mejor vivienda que hemos considerado:
numero de baños completos por planta = 3
numero de asesos por planta = 3
numero de dormitorios = 4
numero de cocinas = 2 
y el precio de la casa. 


### Pista:
#### Calcula las variables: 
- Número de servicios excelentes
- Número de servicios buenos
...

#### Tendréis que tener en cuenta también variables como número de baños Full Bath, cocinas Kitchen AbvGr o dormitorios Bedroom AbvGr. Generando por ejemplo (número de estas variables por encima de media)

#### Finalmente precio de venta

FIN
